In [1]:
from __future__ import absolute_import, division, print_function

# Feature engineering

Here I will extract and engineer the features that will be the input for a subsequent model.

The main data are of the follwoing format:

* The data report weather measurements for ~1000 unique locations in ~150 counties across 5 states. 
* Most locations have one entry per day reporting the current weather conditions.
* At the end of the season, the harvest generates a certain yield. This yield is propagated to *all* entries in the data set, even though it is only a final value.
* The reported yield number refers to the yield in the county and is not specific to a location. It is unclear if it is an average or a sum for the county.



## Goal

My goal here is to create a profile across the season for each location from the avaiable plus additional data. This will leave me with one set of feature values for each location, which is connected to a final yield. And that will be the input to my model.

## Imports

In [2]:
import os
import pickle
import time

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap


%matplotlib inline

## Functions

In [11]:
def find_30day_extreme(in_df, in_column, in_agg='min'):
    """
    Calculates a rolling window mean. Then determines the min/max/mean 
    value of all the window aggregates.
    Window size: 30 days.
    in_df: input dataframe
    in_column: column to be aggregated
    in_agg: aggregation function for the list of results ('min','max','mean')
    """
    # Pandas 30 day rolling window. Make sure there are at least 10 samples in the window.
    agg_df = in_df[[in_column,'Date']].rolling('30d', on='Date', min_periods=10).mean()
    agg_list = agg_df[in_column]
    if in_agg == 'min':
        return np.min(agg_list)
    if in_agg == 'max':
        return np.max(agg_list)
    if in_agg == 'mean':
        return np.mean(agg_list)
 


In [12]:
def calculate_features(tmp_df):
    result = []
    # Minimum average temperature in 30-day period
    result.append(find_30day_extreme(tmp_df, 'temperatureAverage', in_agg='min'))
    # Maximum average temperature in 30-day period
    result.append(find_30day_extreme(tmp_df, 'temperatureAverage', in_agg='max'))
    # Minimum NDVI in 30-day period
    tmp1 = find_30day_extreme(tmp_df, 'temperatureAverage', in_agg='min')
    # Maximum NDVI in 30-day period
    tmp2 = find_30day_extreme(tmp_df, 'temperatureAverage', in_agg='max')
    result.append(tmp2/tmp1)
    # Mean temperature difference and variance
    result.append(tmp_df['temperatureDiff'].mean())
    result.append(tmp_df['temperatureDiff'].std())
    # Mean wind speed
    result.append(tmp_df['windSpeed'].mean())
    # Total precipitation
    result.append(tmp_df['precipTotal'].max())
    # Total yield
    result.append(tmp_df['Yield'].max())
    #
    return result


## Data

These data have been slighlty pre-processed. See 01_data_exploration.ipynb for details.

In [5]:
cwd = os.getcwd()
df_2013 = pd.read_pickle(os.path.join(cwd,'data','df_2013_clean.df'))
df_2014 = pd.read_pickle(os.path.join(cwd,'data','df_2014_clean.df'))


## Additional data

I have also obtained information on the elevation and length_of_day for each location (see 03_elevation_and_length_of_day.ipynb).

In [6]:
# Load data - dictionaries
with open(os.path.join('data','elevation.pickle'), 'rb') as handle:
    elevation = pickle.load(handle)
with open(os.path.join('data','length_of_day.pickle'), 'rb') as handle:
    length_of_day = pickle.load(handle)


## Features

Each location comes with:

* longitude
* latitude

The features I am going to engineer for each location are:

* the total amount of precipitation
* the minimum average temperature in a consecutive 30-day period
* the maximum average temperature in a consecutive 30-day period
* ratio of maximum average NDVI in a consecutive 30-day period and its respective minimum
* the mean temperature difference between daily min/max temperatures
* the standard deviation of the above mean
* the total average wind speed

I will add to those features external values of:

* the hours of daylight
* the elevation

During the engineering I will keep both years (2013 and 2014) separate. Even though these data cover mostly the same locations (~>80% overlap), weather conditions and yield is likely to be different. Keeping them separate provides additional data points. 

## Exclude locations with only very few measurements across the growing period

Since I am aggregating data across the growing period, I will for this current approach remove locations with only very few records. For these locations it is difficult to generate the features above in a straight forward manner. (See 01_data_exploration.ipynb for details on how these locations were identified).

In [7]:
locs = df_2013['Location'].unique().tolist()
print(len(locs))
n_records_2013 = df_2013.groupby(by='Location').agg({'Location': {'Count' : lambda x: x.count()}})
n_records_2013['Location']['Count'].value_counts().sort_index(ascending=False)
klocs = n_records_2013[n_records_2013['Location']['Count'] >= 153].index.get_level_values('Location').values
locs_2013 = df_2013['Location'][df_2013['Location'].isin(klocs)].unique().tolist()
print(len(locs_2013))
#
locs = df_2014['Location'].unique().tolist()
print(len(locs))

n_records_2014 = df_2014.groupby(by='Location').agg({'Location': {'Count' : lambda x: x.count()}})
n_records_2014['Location']['Count'].value_counts().sort_index(ascending=False)
klocs = n_records_2014[n_records_2014['Location']['Count'] >= 153].index.get_level_values('Location').values
locs_2014 = df_2014['Location'][df_2014['Location'].isin(klocs)].unique().tolist()
print(len(locs_2014))



1014
955
1035
982


## 2013

In [8]:
df_2013.head()

,CountyName,State,Latitude,Longitude,Date,cloudCover,dewPoint,humidity,precipIntensity,precipProbability,...,windBearing,windSpeed,NDVI,DayInSeason,Yield,Location,precipTotal,temperatureDiff,temperatureRatio,temperatureAverage
0,Adams,Washington,46.811686,-118.695237,2013-11-30,0.00,29.53,0.91,0.0000,0.00,...,214,1.18,134.110657,0,35.7,"(-118.6952372, 46.8116858)",0.000,8.22,1.299127,31.590
1,Adams,Washington,46.929839,-118.352109,2013-11-30,0.00,29.77,0.93,0.0001,0.05,...,166,1.01,131.506592,0,35.7,"(-118.3521093, 46.9298391)",0.000,8.18,1.303863,31.010
2,Adams,Washington,47.006888,-118.510160,2013-11-30,0.00,29.36,0.94,0.0001,0.06,...,158,1.03,131.472946,0,35.7,"(-118.5101603, 47.0068881)",0.020,6.43,1.238590,30.165
3,Adams,Washington,47.162342,-118.699677,2013-11-30,0.91,29.47,0.94,0.0002,0.15,...,153,1.84,131.288300,0,35.7,"(-118.6996774, 47.1623419)",0.036,6.02,1.221568,30.180
4,Adams,Washington,47.157512,-118.434056,2013-11-30,0.91,29.86,0.94,0.0003,0.24,...,156,1.85,131.288300,0,35.7,"(-118.4340559, 47.157512)",0.000,6.78,1.250462,30.460


In [9]:
features = ['longitude',
            'latitude',
            'elevation',
            'LOD',
            'total_precipitation',
            'minMAT30',
            'maxMAT30',
            'ratioMNDVI30',
            'mean_wind_speed',
            'mean_temperature_diff',
            'std_temperature_diff',
            'yield',
           ]

df_2013_new = pd.DataFrame(columns=features)

In [13]:
now = time.time()

for idx,loc in enumerate(locs_2013):
    tmp_df = df_2013[df_2013['Location'] == loc]
    (min_mean_average_temparature_30days, 
     max_mean_average_temparature_30days,
     ratio_mean_ndvi_30days,
     mean_temperature_diff,
     std_temperature_diff,
     mean_wind_speed,
     total_precipitation,
     total_yield) = calculate_features(tmp_df)
    try:
        new_elevation = elevation[loc]
    except:
        print('Elevation: no match for location found', loc)
    try:
        new_length_of_day = length_of_day[loc]
    except:
        print('Length-of-day: no match for location found', loc)
    longitude = loc[0]
    latitude = loc[1]
    observations = {'longitude': longitude, 
        'latitude': latitude,
        'elevation': new_elevation,
        'LOD': new_length_of_day,
        'total_precipitation': total_precipitation,
        'minMAT30': min_mean_average_temparature_30days,
        'maxMAT30': max_mean_average_temparature_30days,
        'ratioMNDVI30': ratio_mean_ndvi_30days,
        'mean_wind_speed': mean_wind_speed,
        'mean_temperature_diff': mean_temperature_diff,
        'std_temperature_diff': std_temperature_diff,
        'yield': total_yield,
       }
    df_2013_new.loc[idx] = pd.Series(observations)
        
print('Exec. time: {:5.2f} s'.format(time.time()-now))

Exec. time: 37.78 s


In [14]:
len(locs_2013)

955

In [15]:
df_2013_new.shape

(955, 12)

In [16]:
df_2013_new.head(10)

,longitude,latitude,elevation,LOD,total_precipitation,minMAT30,maxMAT30,ratioMNDVI30,mean_wind_speed,mean_temperature_diff,std_temperature_diff,yield
0,-118.695237,46.811686,427.179199,11.171944,9.982,22.090000,62.033000,2.808194,5.298548,17.294409,9.061316,35.7
1,-118.352109,46.929839,524.561829,11.168056,12.312,21.140909,60.183833,2.846795,5.594247,16.485699,8.485870,35.7
2,-118.510160,47.006888,499.476074,11.165833,13.750,20.644545,59.363000,2.875481,6.132742,16.913817,8.444363,35.7
3,-118.699677,47.162342,521.150330,11.160833,12.934,20.151818,59.682000,2.961619,6.120430,16.800269,8.512496,35.7
4,-118.434056,47.157512,571.114075,11.161111,16.644,20.138182,58.143167,2.887210,6.055968,16.077849,8.201818,35.7
5,-118.958859,47.150327,451.084656,11.161111,8.905,21.740833,61.464167,2.827130,5.320806,17.446237,8.471867,35.7
6,-98.330851,36.995835,391.887451,11.439722,47.256,28.731667,71.526500,2.489466,9.218011,24.476720,9.612832,14.4
7,-98.404629,36.988813,401.383514,11.439722,51.077,28.703833,71.522333,2.491735,9.210806,24.486882,9.634120,14.4
8,-98.319893,36.702452,359.771393,11.446389,55.124,29.362167,72.147500,2.457159,9.504355,24.687957,9.656204,14.4
9,-98.539816,36.628781,413.238281,11.448056,67.264,29.471167,72.077000,2.445679,9.398011,24.507634,9.583744,14.4


## 2014

In [17]:
df_2014.head()

,CountyName,State,Latitude,Longitude,Date,cloudCover,dewPoint,humidity,precipIntensity,precipProbability,...,windBearing,windSpeed,NDVI,DayInSeason,Yield,Location,precipTotal,temperatureDiff,temperatureRatio,temperatureAverage
0,Adams,Washington,46.929839,-118.352109,2014-11-30,0.00,6.77,0.69,0.0,0.0,...,9,3.80,136.179718,0,35.6,"(-118.3521093, 46.9298391)",0.0,16.97,3.438218,15.445
1,Adams,Washington,47.150327,-118.958859,2014-11-30,0.00,6.66,0.65,0.0,0.0,...,352,6.03,135.697540,0,35.6,"(-118.9588592, 47.1503267)",0.0,17.17,2.971297,17.295
2,Adams,Washington,46.811686,-118.695237,2014-11-30,0.00,6.55,0.67,0.0,0.0,...,25,3.59,135.676956,0,35.6,"(-118.6952372, 46.8116858)",0.0,16.41,2.986683,16.465
3,Adams,Washington,47.162342,-118.699677,2014-11-30,0.03,7.32,0.69,0.0,0.0,...,1,5.18,135.005798,0,35.6,"(-118.6996774, 47.1623419)",0.0,17.38,3.145679,16.790
4,Adams,Washington,47.157512,-118.434056,2014-11-30,0.04,7.62,0.70,0.0,0.0,...,5,4.69,134.803864,0,35.6,"(-118.4340559, 47.157512)",0.0,16.51,2.984375,16.575


In [18]:
features = ['longitude',
            'latitude',
            'elevation',
            'LOD',
            'total_precipitation',
            'minMAT30',
            'maxMAT30',
            'ratioMNDVI30',
            'mean_wind_speed',
            'mean_temperature_diff',
            'std_temperature_diff',
            'yield',
           ]

df_2014_new = pd.DataFrame(columns=features)

In [19]:
now = time.time()

for idx,loc in enumerate(locs_2014):
    tmp_df = df_2014[df_2014['Location'] == loc]
    (min_mean_average_temparature_30days, 
     max_mean_average_temparature_30days,
     ratio_mean_ndvi_30days,
     mean_temperature_diff,
     std_temperature_diff,
     mean_wind_speed,
     total_precipitation,
     total_yield) = calculate_features(tmp_df)
    try:
        new_elevation = elevation[loc]
    except:
        print('Elevation: no match for location found', loc)
    try:
        new_length_of_day = length_of_day[loc]
    except:
        print('Length-of-day: no match for location found', loc)
    longitude = loc[0]
    latitude = loc[1]
    observations = {'longitude': longitude, 
        'latitude': latitude,
        'elevation': new_elevation,
        'LOD': new_length_of_day,
        'total_precipitation': total_precipitation,
        'minMAT30': min_mean_average_temparature_30days,
        'maxMAT30': max_mean_average_temparature_30days,
        'ratioMNDVI30': ratio_mean_ndvi_30days,
        'mean_wind_speed': mean_wind_speed,
        'mean_temperature_diff': mean_temperature_diff,
        'std_temperature_diff': std_temperature_diff,
        'yield': total_yield,
       }
    df_2014_new.loc[idx] = pd.Series(observations)
        
print('Exec. time: {:5.2f} s'.format(time.time()-now))

Exec. time: 39.80 s


In [20]:
len(locs_2014)

982

In [22]:
df_2014_new.shape

(982, 12)

In [23]:
df_2014_new.head(10)

,longitude,latitude,elevation,LOD,total_precipitation,minMAT30,maxMAT30,ratioMNDVI30,mean_wind_speed,mean_temperature_diff,std_temperature_diff,yield
0,-118.352109,46.929839,524.561829,11.168056,3.362,29.588000,62.185833,2.101725,4.202903,17.679355,9.148266,35.6
1,-118.958859,47.150327,451.084656,11.161111,1.536,29.681000,63.494667,2.139236,4.044194,17.873925,9.295265,35.6
2,-118.695237,46.811686,427.179199,11.171944,2.472,30.269500,63.789667,2.107391,3.991398,18.457634,9.581113,35.6
3,-118.699677,47.162342,521.150330,11.160833,3.528,29.487500,62.002333,2.102665,4.630054,17.826935,9.166665,35.6
4,-118.434056,47.157512,571.114075,11.161111,4.317,28.834833,60.857667,2.110561,4.717097,17.187527,8.905124,35.6
5,-118.510160,47.006888,499.476074,11.165833,3.671,29.496000,61.706000,2.092012,4.679677,17.844516,9.182010,35.6
6,-98.454318,36.508221,394.365906,11.451111,10.565,31.529167,65.112000,2.065135,8.180699,20.610538,9.833779,29.3
7,-98.444745,36.650698,401.645874,11.447778,13.368,31.417333,65.034000,2.070004,8.170323,20.738495,9.912246,29.3
8,-98.319893,36.702452,359.771393,11.446389,12.767,31.339000,65.082167,2.076715,8.287366,20.991828,10.070211,29.3
9,-98.539816,36.628781,413.238281,11.448056,14.993,31.370833,65.004333,2.072126,8.169570,20.766344,9.928065,29.3


In [24]:
df_2013.columns

Index([u'CountyName', u'State', u'Latitude', u'Longitude', u'Date',
       u'cloudCover', u'dewPoint', u'humidity', u'precipIntensity',
       u'precipProbability', u'precipAccumulation', u'precipTypeIsRain',
       u'precipTypeIsSnow', u'pressure', u'temperatureMax', u'temperatureMin',
       u'visibility', u'windBearing', u'windSpeed', u'NDVI', u'DayInSeason',
       u'Yield', u'Location', u'precipTotal', u'temperatureDiff',
       u'temperatureRatio', u'temperatureAverage'],
      dtype='object')

In [25]:
df_2014.columns

Index([u'CountyName', u'State', u'Latitude', u'Longitude', u'Date',
       u'cloudCover', u'dewPoint', u'humidity', u'precipIntensity',
       u'precipProbability', u'precipAccumulation', u'precipTypeIsRain',
       u'precipTypeIsSnow', u'pressure', u'temperatureMax', u'temperatureMin',
       u'visibility', u'windBearing', u'windSpeed', u'NDVI', u'DayInSeason',
       u'Yield', u'Location', u'precipTotal', u'temperatureDiff',
       u'temperatureRatio', u'temperatureAverage'],
      dtype='object')

## Save features to disk

In [26]:
df_2013_new.to_pickle(os.path.join('data','df_2013_features.df'))
df_2014_new.to_pickle(os.path.join('data','df_2014_features.df'))
